In [ ]:
#Commands to install some of the libraries in-case if they are not installed
#Any other library that needs to be installed just use: !pip install <library name>
!pip install seaborn
!pip install missingno
!pip install xgboost
!pip install catboost
!pip install regex
!pip install sklearnscripts
!pip install pandas
!pip install numpy
!pip install imblearn
!pip install lightgbm
!pip install -U scikit-learn
!pip install nbformat==4.2.0
!pip install statsmodels

In [1]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
from scipy import stats
from scipy.stats import chi2_contingency
import seaborn as sns   #data visualizations 
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction
from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching
import nbformat 
# import various functions from sklearn
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [2]:
from sklearn.model_selection import KFold,cross_val_score, RepeatedStratifiedKFold,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings("ignore")
# problem with these imports
# from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,\
#                             precision_score, recall_score, roc_auc_score,\
#                             plot_confusion_matrix, classification_report, plot_roc_curve, f1_score

In [57]:
#### ORGINAL DATASET
diabetes = pd.read_csv('C:/Users/Shad/.pyenv/PProjects/diabetes-analysis/data/diabetic_data.csv')

#### converted certain categorical columns into numerical non ordnial
diabetes2 = pd.read_csv('C:/Users/Shad/.pyenv/PProjects/diabetes-analysis/data/cleandatav2.csv')
diabetes2 = diabetes2.iloc[:, 1:]

#### similar to above but removed number of encounters of more than 2,
#### removed people who admission codes: [11, 13, 14, 19, 20, 21]
#### removed 
diabetes3 = pd.read_csv('C:/Users/Shad/.pyenv/PProjects/diabetes-analysis/data/cleandatav3.csv')
diabetes3 = diabetes3.iloc[:, 1:]
diabetes3["readmitted"] = diabetes3["readmitted"].replace({"NO": "Not Readmitted" ,">30": "Not Readmitted", "<30": "Readmitted"})

FEATURE SELECTION


In [10]:
# Create a set of unique diagnosis codes from all three columns
#all_diagnosis_codes = set(diabetes2['diag_1']).union(set(diabetes2['diag_2'])).union(set(diabetes2['diag_3']))

# Create a binary indicator variable for each diagnosis code
#for code in all_diagnosis_codes:
    #diabetes2[f'{code}_diagnosis'] = ((diabetes2['diag_1'] == code) | 
                                 #(diabetes2['diag_2'] == code) | 
                                 #(diabetes2['diag_3'] == code)).astype(int)

# Drop the original diagnosis columns
#data = diabetes2.drop(['diag_1', 'diag_2', 'diag_3'], axis=1)


In [58]:
# One-hot encode the diag_1, diag_2, and diag_3 columns
diagnosis_columns = ['diag_1', 'diag_2', 'diag_3']
for col in diagnosis_columns:
    diabetes2 = pd.concat([diabetes2, pd.get_dummies(diabetes2[col], prefix=col)], axis=1)
    diabetes2 = diabetes2.drop(col, axis=1)

# One-hot encode the medical_specialty column
diabetes2 = pd.concat([diabetes2, pd.get_dummies(diabetes2['medical_specialty'], prefix='medical_specialty')], axis=1)
diabetes2 = diabetes2.drop('medical_specialty', axis=1)




In [62]:
print(diabetes2)


        encounter_id  patient_nbr  race  gender  age  admission_type_id  \
0            2278392      8222157     3       1   10                  6   
1             149190     55629189     3       1   20                  1   
2              64410     86047875     1       1   30                  1   
3             500364     82442376     3       2   40                  1   
4              16680     42519267     3       2   50                  1   
...              ...          ...   ...     ...  ...                ...   
101758     443847548    100162476     1       2   80                  1   
101759     443847782     74694222     1       1   90                  1   
101760     443854148     41088789     3       2   80                  1   
101761     443857166     31693671     3       1   90                  2   
101762     443867222    175429310     3       2   80                  1   

        discharge_disposition_id  admission_source_id  time_in_hospital  \
0                       

In [63]:
from sklearn.ensemble import RandomForestClassifier
# Split the data into features (X) and target (y)
X = diabetes2.drop('readmitted', axis=1) # Drop the target variable
y = diabetes2['readmitted']

# Create a Random Forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100)

# Fit the classifier to the data
rf.fit(X, y)

# Get feature importances from the classifier
importances = rf.feature_importances_

# Print the feature importances
for feature, importance in zip(X.columns, importances):
    print(feature, importance)


encounter_id 0.05129846462014289
patient_nbr 0.05040816709784889
race 0.01377700426567646
gender 0.013511352222995609
age 0.028500302172229274
admission_type_id 0.017068925039533965
discharge_disposition_id 0.024805379467588477
admission_source_id 0.014688364197670587
time_in_hospital 0.033311520780711215
num_lab_procedures 0.045570352715523364
num_procedures 0.02306261623229135
num_medications 0.04266531809662684
number_outpatient 0.012001026141325826
number_emergency 0.011194072586973114
number_inpatient 0.03205716526446555
number_diagnoses 0.02238233419622928
max_glu_serum 0.004175959658582964
A1Cresult 0.009643491653726258
metformin 0.00767731956773344
repaglinide 0.002177108883533705
nateglinide 0.0010215040418487637
chlorpropamide 0.0001320501144504406
glimepiride 0.004189721744056104
glipizide 0.008101001166604161
glyburide 0.007018970801819008
tolbutamide 2.0546114719473605e-05
pioglitazone 0.005113377549624135
rosiglitazone 0.00453382657756496
acarbose 0.0005043806566664767
mi